# Import Modules

In [0]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import random
import numpy as np
import torchvision
import matplotlib.pyplot as plt
from hyperopt import fmin, tpe, hp
import time
import copy
import os

# Check Available Devices


In [2]:
if torch.cuda.is_available:
  avDev = torch.device("cuda")
else:
  avDev = torch.device("cpu")
print(avDev)

cuda


# Set Seed

In [0]:
seed = 1
torch.manual_seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic=True

# Utility for Displaying Images, Error Curve

In [0]:
def showImagesWithLabels(img,labels):
    img = img / 2 + 0.5     # unnormalize
    img = img.numpy()
    plt.imshow(np.transpose(img, (1, 2, 0)))
    plt.show()

In [0]:
def plot_error_curve(errors):
  plt.suptitle('Learning Curve', fontsize=20)
  plt.xlabel('Iterations', fontsize=18)
  plt.ylabel('Classification Error', fontsize=16)
  plt.plot(np.array(errors))

In [0]:
def showSegmentedImage(colorMapList):  #Hopefully this should work

# Load Data Functions

In [0]:
def transforms(listTransforms):
    transforms.Compose(listTransforms)

In [0]:
def loadDataSet(dirName,toshuffle):
#Only for the labelled image data
imagenet_data = torchvision.datasets.ImageNet(dirName)
data_loader = torch.utils.data.DataLoader(imagenet_data,
                                          shuffle=toshuffle)

# Hyperparameter

In [0]:
lr
batchSize
epoch
optimizer


# Load Data And Make Iterable

In [9]:
listTransforms = [transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))]
transforms(listTransforms)
loadDataSet(dirName,toShuffle)#Here specify train and test

TypeError: ignored

In [0]:
dataiter = train_loader.__iter__()#whichever loaded
images, labels = dataiter.next()

# Model Definition

In [0]:
class ResNet18(nn.Module):
    def __init__(self, original_model, outputs_indices):
        super(ResNet18, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-2])
        self.outputs_indices = [0] + outputs_indices
        print(self.outputs_indices)

    def forward(self, x):
        out = []

        for i in range(len(self.outputs_indices) - 1):
            x = self.features[self.outputs_indices[i]:self.outputs_indices[i + 1]](x)
            out.append(x)
        return out


class soccerSegment(nn.ModuleList):
    def __init__(self, resnet18, outputs_indices, skips_arch, deconvs_arch, bn_arch, last_layer_arch):
        super(soccerSegment, self).__init__()
        self.resnet18 = ResNet18(resnet18, outputs_indices)
        
        # skips_arch = [64, 128, 256, 256, 0]
        # deconvs_arch = [512, 256, 256, 128]
        # bn_arch = [512, 512, 256]
        # last_layer_arch = 256
        self.skips = nn.ModuleList(
            [nn.Conv2d(skips_arch[i], skips_arch[i + 1], kernel_size=1, stride=1, padding=0) for i in
             range(len(skips_arch) - 2)])

        self.deconvs = nn.ModuleList(
            reversed([nn.ConvTranspose2d(deconvs_arch[i] + skips_arch[len(skips_arch) - i - 1], deconvs_arch[i + 1],
                                         kernel_size=2, stride=2, padding=0) for i in
                      range(len(deconvs_arch) - 1)]))

        self.bns = nn.ModuleList(
            [nn.BatchNorm2d(num_features=bn_arch[i]) for i in
             reversed(range(len(bn_arch)))])
        self.relu = nn.ReLU()

        self.conv_det = nn.Conv2d(last_layer_arch, 3, kernel_size=1, stride=1, padding=0)
        self.conv_seg = nn.Conv2d(last_layer_arch, 3, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        skip_links = self.resnet18(x)

        for i in reversed(range(len(skip_links))):
            if i == len(skip_links) - 1:
                skip_links[i - 1] = torch.cat(
                    (self.skips[i - 1](skip_links[i - 1]), self.deconvs[i - 1](self.relu(skip_links[i]))),
                    1)
            elif i == 0:
                skip_links[i] = self.relu(self.bns[i](skip_links[i]))
            else:
                skip_links[i - 1] = torch.cat(
                    (self.skips[i - 1](skip_links[i - 1]),
                     self.deconvs[i - 1](self.relu(self.bns[i](skip_links[i])))),
                    1)
        seg = self.conv_seg(skip_links[i])
        det = self.conv_det(skip_links[i])
        return seg, det

# Training Cycle

In [0]:
def train(args):   #Consist of all args from hyperopt
    
    return loss.item()

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    return model

# Hyperopt

In [0]:
optimizers = [torch.optim.SGD, torch.optim.Adam, torch.optim.Adagrad, torch.optim.Adadelta, torch.optim.RMSprop]
regularizers = ['l1','l2']
space = [hp.choice('number_of_layers',list_of_layers),hp.choice('optimizer',optimizers),hp.choice('regularizer',regularizers)]
best_classifier = fmin(train,space,algo=tpe.suggest,max_evals=30)
optimizer = optimizers[int(best_classifier['optimizer'])]



In [0]:
#Use the best found optimizer and criterion
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=25)

# Visualization of Results

In [0]:
#I think this got covered 

# Metric Measures